<a href="https://colab.research.google.com/github/GHROTH-L/crawler/blob/main/YAHOO%E8%82%A1%E5%83%B9%E7%88%AC%E8%9F%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [128]:
import time
import requests
import csv
from bs4 import BeautifulSoup
#YAHOO 網址
url = "https://tw.stock.yahoo.com/q/q?s=" #s= 後面是要放股票號碼

In [123]:
#產生搜尋的url 以及股票名稱
def gen_url(url, stocks):
  urls = []
  for stock in stocks:
    urls.append(url+stock)
  return urls

In [4]:
#get_resouse 抓取資料
def get_r(url):
  headers = {"user-agen" :"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"}
  return requests.get(url,headers= headers)

In [120]:
#製造stock_crawler
# stock_id name stauts price yclose volume high low
def stock_crawler(soup, stock_id):
  sp3 = soup.find_all(text="成交")[0].parent.parent.parent
  name = soup.find_all("h1")[1].text
  price = sp3.select("span[class='Fw(600) Fz(16px)--mobile Fz(14px) D(f) Ai(c) C($c-trend-up)']")[0].text
  yclose = sp3.select("span[class='Fw(600) Fz(16px)--mobile Fz(14px) D(f) Ai(c)']")[0].text
  volume = sp3.select("span[class='Fw(600) Fz(16px)--mobile Fz(14px)']")[1].text
  high = sp3.select("span[class='Fw(600) Fz(16px)--mobile Fz(14px) D(f) Ai(c) C($c-trend-up)']")[2].text
  low = sp3.select("span[class='Fw(600) Fz(16px)--mobile Fz(14px) D(f) Ai(c) C($c-trend-up)']")[3].text
  if float(price) > float(yclose):
    status = "漲" + sp3.select("span[class='Fw(600) Fz(16px)--mobile Fz(14px) D(f) Ai(c) C($c-trend-up)']")[5].text
  else:
    status = "跌" + sp3.select("span[class='Fw(600) Fz(16px)--mobile Fz(14px) D(f) Ai(c) C($c-trend-up)']")[5].text
  return [stock_id,name,status,price,yclose,volume,high,low]


In [70]:
#stock_soup 製造soup
'''
代碼 名稱 漲跌幅 股價 昨收 總量 最高 最低
'''
def stock_soup(urls):
  stocks= [["代碼","名稱","漲跌幅","股價","昨收","總量","最高","最低"]]
  
  for url in urls:
    stock_id = url.split("=")[-1] 
    '''
    例如，如果url的值是"https://example.com?id=12345"，那么url.split("=")将返回["https://example.com?id", "12345"]。[-1]是通过索引访问列表中的最后一个元素。在上述示例中，url.split("=")[-1]将返回"12345"。
    '''
    print("抓取:"+stock_id+"網路資料...")
    r= get_r(url)
    if r.status_code == requests.codes.ok:
      soup = BeautifulSoup(r.text,"html.parser") #製造soup
      stock = stock_crawler(soup, stock_id)
      stocks.append(stock)
      print("wait for 5 seconds")
      time.sleep(5)
    else:
      print("請求錯誤")
  return stocks



In [133]:
if __name__ == "__main__":  #主程式開頭
  urls = gen_url(url, ["3711","2330"])
  #print(urls) 測試連結
  stocks = stock_soup(urls)
  for stock in stocks:
    print(stock)

抓取:3711網路資料...


<ipython-input-120-40d728c7c43a>:4: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  sp3 = soup.find_all(text="成交")[0].parent.parent.parent


wait for 5 seconds
抓取:2330網路資料...
wait for 5 seconds
['代碼', '名稱', '狀態', '股價', '昨收', '總量', '最高', '最低']
['3711', '日月光投控', '漲2.0', '110.5', '108.5', '42,112', '111.5', '109.0']
['2330', '台積電', '漲23', '566', '543', '95,692', '568', '563']
